In [3]:
import numpy as np
import plotly.graph_objects as go
import pandas as pd
from typing import Tuple, Optional

# --- 1. 幾何布朗運動函數 (核心數學模型) ---
def geometric_brownian_motion(
    S0: float, mu: float, sigma: float, dt: float, T: float, num_simulations: int
) -> np.ndarray:
    """
    生成幾何布朗運動的路徑。
    """
    num_steps = int(T / dt)
    dW = np.random.normal(loc=0.0, scale=1.0, size=(num_simulations, num_steps)) * np.sqrt(dt)

    price_paths = np.zeros((num_simulations, num_steps + 1))
    price_paths[:, 0] = S0

    for t in range(1, num_steps + 1):
        price_paths[:, t] = price_paths[:, t-1] * np.exp(
            (mu - 0.5 * sigma**2) * dt + sigma * dW[:, t-1]
        )
    return price_paths

# --- 2. 數據轉換函數：將連續價格轉換為 K 線 (OHLC) 格式 ---
def convert_to_ohlc(
    price_path: np.ndarray, 
    k_period: int,          # K線週期
    dt: float,              # 時間步長
    origin_date: str = '2025-01-01' # K線圖的起始日期
) -> pd.DataFrame:
    """將單一價格路徑轉換為 K 線數據 (OHLC 格式)，並創建日期索引。"""
    
    daily_prices = pd.Series(price_path[1:])
    
    # 確保數據長度是 k_period 的倍數以便分組
    num_groups = len(daily_prices) // k_period
    group_labels = np.repeat(np.arange(num_groups), k_period)
    daily_prices = daily_prices.iloc[:len(group_labels)]
    
    grouped_data = daily_prices.groupby(group_labels)
    
    ohlc = pd.DataFrame()
    
    # Open (開盤價): 設為前一個週期的 Close
    ohlc['Open'] = grouped_data.first().shift(1)
    ohlc.iloc[0, ohlc.columns.get_loc('Open')] = price_path[0] # 第一個 Open 使用 S0
    
    # Close, High, Low
    ohlc['Close'] = grouped_data.last()
    ohlc['High'] = grouped_data.max()
    ohlc['Low'] = grouped_data.min()
    
    # --- 關鍵修正時間軸邏輯 ---
    
    # 1. 創建數值型時間步長陣列，代表累積的**交易日**數
    # 假設 k_period = 5，則 K 線的結束點是 5, 10, 15... 交易日
    days_elapsed = (ohlc.index.values + 1) * k_period
    
    # 2. 將數值型的天數傳入 pd.to_datetime，並指定 unit='D'，搭配 origin
    # 這樣就避免了 'timedelta64[ns]' 與 'origin' 不兼容的問題
    ohlc['Date'] = pd.to_datetime(days_elapsed, unit='D', origin=origin_date)
    
    # 重新整理欄位順序
    ohlc = ohlc[['Date', 'Open', 'High', 'Low', 'Close']]
    
    return ohlc

# --- 3. 主程序執行 ---
if __name__ == '__main__':
    # --- 模擬參數設定 ---
    S0 = 100.0        # 初始股價
    mu = 0.08         # 年化報酬率 (8%)
    sigma = 0.25      # 年化波動率 (25%)
    T = 2.0           # 模擬兩年
    dt = 1/252        # 時間步長，一個交易日
    num_simulations = 1 # 只需要一條路徑來繪製 K 線
    k_period = 5        # K線週期：每 5 個交易日合成一根 K 線（週 K 線）
    origin_date = '2025-01-01' # 模擬的起始日期

    # --- 執行模擬 ---
    simulated_prices = geometric_brownian_motion(S0, mu, sigma, dt, T, num_simulations)
    
    # 選擇第一條路徑進行 K 線轉換
    single_path = simulated_prices[0, :]
    
    # --- 轉換為 OHLC K 線數據 ---
    ohlc_data = convert_to_ohlc(single_path, k_period, dt, origin_date)
    
    print(f"--- 模擬參數 ---")
    print(f"初始價格 (S0): {S0}")
    print(f"年化報酬率 (μ): {mu*100:.2f}%")
    print(f"年化波動率 (σ): {sigma*100:.2f}%")
    print(f"K線週期: {k_period} 日")
    print(f"總 K 線數: {len(ohlc_data)}")
    print("\n--- K 線數據 (前 5 筆) ---")
    print(ohlc_data.head())
    
    # --- 4. 使用 Plotly 繪製 K 線圖 ---
    fig = go.Figure(data=[go.Candlestick(
        x=ohlc_data['Date'],
        open=ohlc_data['Open'],
        high=ohlc_data['High'],
        low=ohlc_data['Low'],
        close=ohlc_data['Close'],
        name=f'{k_period} 日 K 線'
    )])

    fig.update_layout(
        title={
            'text': f'幾何布朗運動模擬 K 線圖 (μ={mu*100}%, σ={sigma*100}%, {k_period}日週期)',
            'y':0.9, 'x':0.5, 'xanchor': 'center', 'yanchor': 'top'
        },
        xaxis_title='日期',
        yaxis_title='股票價格',
        xaxis_rangeslider_visible=False,
        template='plotly_white'
    )
    fig.show()

--- 模擬參數 ---
初始價格 (S0): 100.0
年化報酬率 (μ): 8.00%
年化波動率 (σ): 25.00%
K線週期: 5 日
總 K 線數: 100

--- K 線數據 (前 5 筆) ---
        Date        Open       High        Low      Close
0 2025-01-06  100.000000  97.531582  92.922180  92.922180
1 2025-01-11   97.531582  96.389636  91.871956  96.389636
2 2025-01-16   92.962945  98.552370  95.010317  95.421093
3 2025-01-21   96.678537  93.619111  91.590436  92.780603
4 2025-01-26   93.189081  93.212844  92.058148  92.058148
